In [1]:
#goals in this project
#set up a black scholes pricing model
#calculate greeks based on black scholes
#come up with some manual test examples for both BS and Greeks
#test BS against simple market data
#test BS and Greeks against more complex market data
#find discerpencies
#explain discrepencies
#find way to exploit discrepencies using trading data